<a href="https://colab.research.google.com/github/faheem-khaskheli/tensorflow_basics/blob/main/Custom_Training_Loop_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
import tensorflow_datasets as tfds
import matplotlib.ticker as nticker 

In [ ]:
from tqdm import tqdm

In [ ]:
train_data, info = tfds.load('fashion_mnist', split='train', with_info=True)
test_data = tfds.load('fashion_mnist', split='test')

Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.1.incompleteLHVGI1/fashion_mnist-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.1.incompleteLHVGI1/fashion_mnist-test.tfrecord


Dataset fashion_mnist downloaded and prepared to /root/tensorflow_datasets/fashion_mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
class_names = ['T-shirt/top',
               'Trouser/pants',
               'Pullover shirt',
               'Dress',
               'Coat',
               'Sandal',
               'Shirt',
               'Sneaker',
               'Bag',
               'Ankel boot']

In [ ]:
def format_image(data):
  image = data['image']
  image = tf.reshape(image, [-1])
  image = tf.cast(image, 'float32')
  image = image / 255.0
  return image, data['label']

In [ ]:
train_data = train_data.map(format_image)
test_data = test_data.map(format_image)

In [ ]:
batch_size = 64
train = train_data.shuffle(buffer_size=1024).batch(batch_size)
test= test_data.batch(batch_size)

In [ ]:
def base_model(classes):
  input_layer = tf.keras.layers.Input(shape=(784))
  x = tf.keras.layers.Dense(64, activation='relu', name='dense_1')(input_layer)
  x = tf.keras.layers.Dense(64, activation='relu', name='dense_2')(x)
  output = tf.keras.layers.Dense(classes, activation='softmax', name='predictions')(x)
  model = tf.keras.Model(inputs=input_layer, outputs=output)
  return model

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

In [ ]:
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
test_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

In [ ]:
def apply_gradient(optimizer, model, x_batch, y_batch):
  with tf.GradientTape() as tape:
    y_pred = model(x_batch)
    loss = loss_object(y_true=y_batch, y_pred=y_pred)
  
  grad = tape.gradient(loss, model.trainable_weights)
  optimizer.apply_gradients(zip(grad, model.trainable_weights))
  return y_pred, loss

In [ ]:
def train_on_batch():
  losses = []
  pbar = tqdm(total=len(list(enumerate(train))),
              position=0,
              leave=True,
              bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt}')
  for step, (x_batch, y_batch) in enumerate(train):
    y_pred, loss = apply_gradient(optimizer, model, x_batch, y_batch)
    losses.append(loss)
    train_acc_metric(y_batch, y_pred)
  return losses
  pbar.set_description("Training loss for step %s: %.4f" % (int(step), float(loss_value)))
  pbar.update()
  return losses

In [ ]:
def perform_validation():
  losses = []
  for x_val, y_val in test:
    y_pred = model(x_val)
    loss = loss_object(y_val, y_pred)
    losses.append(loss)
    test_acc_metric(y_val, y_pred)
  return losses

In [ ]:
model = base_model(10)

epochs = 10
epochs_val_losses, epochs_train_losses = [], []
for epoch in range(epochs):
  print('Start of epoch %d' % (epoch,))

  losses_train = train_on_batch()
  train_acc = train_acc_metric.result()

  losses_val = perform_validation()
  val_acc = test_acc_metric.result()

  losses_train_mean = np.mean(losses_train)
  losses_val_mean = np.mean(losses_val)

  epochs_val_losses.append(losses_val_mean)
  epochs_train_losses.append(losses_train_mean)

  print('\nEpoch %s: Train loss: %.4f Validation Loss: %.4f, Train Accuracy %.4f, Validation Accuracy %.4f' % (epoch, float(losses_train_mean), losses_val_mean, train_acc, val_acc))

  train_acc_metric.reset_states()
  test_acc_metric.reset_states()

Start of epoch 0


  0%|          | 0/938